In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [47]:
df = pd.read_csv('grant_data_imb.csv')
df.head()

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,...,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
0,0,97A,30B,A,321024.0,50.0,321013.0,30.0,291502.0,20.0,...,2563.0,25.0,Yes,>10 to 15,2.0,6.0,3.0,5.0,15.0,3.0
1,0,36D,10A,G,300201.0,100.0,0.0,0.0,0.0,0.0,...,1038.0,1.0,NaN,Less than 0,0.0,3.0,0.0,4.0,0.0,0.0
2,0,317A,30D,NaN,321013.0,100.0,0.0,0.0,0.0,0.0,...,2763.0,25.0,Yes,>5 to 10,4.0,3.0,6.0,25.0,14.0,14.0
3,0,62B,10B,B,321103.0,30.0,321105.0,40.0,321204.0,30.0,...,2848.0,25.0,NaN,Less than 0,1.0,2.0,1.0,0.0,0.0,0.0
4,0,1A,10A,NaN,270603.0,60.0,321205.0,30.0,320603.0,10.0,...,2678.0,25.0,Yes,>5 to 10,5.0,14.0,0.0,9.0,7.0,0.0


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Grant.Status                            4113 non-null   int64  
 1   Sponsor.Code                            3856 non-null   object 
 2   Grant.Category.Code                     3856 non-null   object 
 3   Contract.Value.Band...see.note.A        1953 non-null   object 
 4   RFCD.Code.1                             3853 non-null   float64
 5   RFCD.Percentage.1                       3853 non-null   float64
 6   RFCD.Code.2                             3853 non-null   float64
 7   RFCD.Percentage.2                       3853 non-null   float64
 8   RFCD.Code.3                             3853 non-null   float64
 9   RFCD.Percentage.3                       3853 non-null   float64
 10  RFCD.Code.4                             3853 non-null   floa

Данный датасет имеет 39 колонок и 4113 строчек, и в каждой столбе значений, кроме первого, имеются пропуски
Имеются два типа данных - числовой(float and int) и категориальный(object), все типы данных соответствуют

In [49]:
df.columns

Index(['Grant.Status', 'Sponsor.Code', 'Grant.Category.Code',
       'Contract.Value.Band...see.note.A', 'RFCD.Code.1', 'RFCD.Percentage.1',
       'RFCD.Code.2', 'RFCD.Percentage.2', 'RFCD.Code.3', 'RFCD.Percentage.3',
       'RFCD.Code.4', 'RFCD.Percentage.4', 'RFCD.Code.5', 'RFCD.Percentage.5',
       'SEO.Code.1', 'SEO.Percentage.1', 'SEO.Code.2', 'SEO.Percentage.2',
       'SEO.Code.3', 'SEO.Percentage.3', 'SEO.Code.4', 'SEO.Percentage.4',
       'SEO.Code.5', 'SEO.Percentage.5', 'Person.ID.1', 'Role.1',
       'Year.of.Birth.1', 'Country.of.Birth.1', 'Home.Language.1',
       'Dept.No..1', 'Faculty.No..1', 'With.PHD.1',
       'No..of.Years.in.Uni.at.Time.of.Grant.1',
       'Number.of.Successful.Grant.1', 'Number.of.Unsuccessful.Grant.1',
       'A..1', 'A.1', 'B.1', 'C.1'],
      dtype='object')

Переведем название фичей в нижний регистр и заменим точку на нижнее подчеркивание

In [50]:
df.columns = [col.lower().replace('.', '_') for col in df.columns]

In [51]:
df.columns

Index(['grant_status', 'sponsor_code', 'grant_category_code',
       'contract_value_band___see_note_a', 'rfcd_code_1', 'rfcd_percentage_1',
       'rfcd_code_2', 'rfcd_percentage_2', 'rfcd_code_3', 'rfcd_percentage_3',
       'rfcd_code_4', 'rfcd_percentage_4', 'rfcd_code_5', 'rfcd_percentage_5',
       'seo_code_1', 'seo_percentage_1', 'seo_code_2', 'seo_percentage_2',
       'seo_code_3', 'seo_percentage_3', 'seo_code_4', 'seo_percentage_4',
       'seo_code_5', 'seo_percentage_5', 'person_id_1', 'role_1',
       'year_of_birth_1', 'country_of_birth_1', 'home_language_1',
       'dept_no__1', 'faculty_no__1', 'with_phd_1',
       'no__of_years_in_uni_at_time_of_grant_1',
       'number_of_successful_grant_1', 'number_of_unsuccessful_grant_1',
       'a__1', 'a_1', 'b_1', 'c_1'],
      dtype='object')

Раздели признаки на количественные и категориальные

In [52]:
features_names_categorical = df.select_dtypes(include=['object']).columns
features_names_quantitative = df.select_dtypes(include=['float', 'int']).columns.drop('grant_status')

In [53]:
for col in features_names_quantitative:
    df[col + '_zero'] = df[col].fillna(0)
    df[col] = df[col].fillna(df[col].mean())

Теперь работаем с категориальными признаками

In [54]:
features_names_categorical

Index(['sponsor_code', 'grant_category_code',
       'contract_value_band___see_note_a', 'role_1', 'country_of_birth_1',
       'home_language_1', 'with_phd_1',
       'no__of_years_in_uni_at_time_of_grant_1'],
      dtype='object')

In [55]:
df['sponsor_code'] = df['sponsor_code'].fillna(df['sponsor_code'].mode()[0])
df['grant_category_code'] = df['grant_category_code'].fillna(df['grant_category_code'].mode()[0])
df['contract_value_band___see_note_a'] = df['contract_value_band___see_note_a'].fillna(df['contract_value_band___see_note_a'].mode()[0])
df['role_1'] = df['role_1'].fillna(df['role_1'].mode()[0])
df['with_phd_1'] = df['with_phd_1'].fillna('N/A')
df['no__of_years_in_uni_at_time_of_grant_1'] = df['no__of_years_in_uni_at_time_of_grant_1'].fillna('N/A')
df['country_of_birth_1'] = df['country_of_birth_1'].fillna('N/A')


In [56]:
df['country_of_birth_1'].value_counts()

country_of_birth_1
Australia                 2530
N/A                        666
Great Britain              304
Western Europe             158
Asia Pacific               149
North America              121
Eastern Europe              60
Middle East and Africa      40
The Americas                32
New Zealand                 31
South Africa                22
Name: count, dtype: int64

In [57]:
country_speaking_english = ['Australia', 'Great Britain', 'North America', 'New Zealand', 'South Africa', 'North America']

is_eanglish_speaker = df['country_of_birth_1'].isin(country_speaking_english)
df.loc[is_eanglish_speaker, 'home_language_1'] = df.loc[is_eanglish_speaker, 'home_language_1'].fillna('English')

df['home_language_1'] = df['home_language_1'].fillna('Other')

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 69 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   grant_status                            4113 non-null   int64  
 1   sponsor_code                            4113 non-null   object 
 2   grant_category_code                     4113 non-null   object 
 3   contract_value_band___see_note_a        4113 non-null   object 
 4   rfcd_code_1                             4113 non-null   float64
 5   rfcd_percentage_1                       4113 non-null   float64
 6   rfcd_code_2                             4113 non-null   float64
 7   rfcd_percentage_2                       4113 non-null   float64
 8   rfcd_code_3                             4113 non-null   float64
 9   rfcd_percentage_3                       4113 non-null   float64
 10  rfcd_code_4                             4113 non-null   floa

Всё, пропуски заполнены

Разделим фичи и целевую переменную

In [59]:
features = df.drop(['grant_status'], axis=1)
target = df['grant_status']

Преобразуем категориальные признаки в количественные с помощью прямого кодирования

In [60]:
features_ohe = pd.get_dummies(features, drop_first=True).astype(int)
features_ohe.head()

,rfcd_code_1,rfcd_percentage_1,rfcd_code_2,rfcd_percentage_2,rfcd_code_3,rfcd_percentage_3,rfcd_code_4,rfcd_percentage_4,rfcd_code_5,rfcd_percentage_5,...,country_of_birth_1_South Africa,country_of_birth_1_The Americas,country_of_birth_1_Western Europe,home_language_1_Other,with_phd_1_Yes,no__of_years_in_uni_at_time_of_grant_1_>5 to 10,no__of_years_in_uni_at_time_of_grant_1_>=0 to 5,no__of_years_in_uni_at_time_of_grant_1_Less than 0,no__of_years_in_uni_at_time_of_grant_1_N/A,no__of_years_in_uni_at_time_of_grant_1_more than 15
0,321024,50,321013,30,291502,20,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,300201,100,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,321013,100,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,321103,30,321105,40,321204,30,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,270603,60,321205,30,320603,10,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


Разделим данные на тренировочную и тестовую часть

In [61]:
features_train, features_val, target_train, target_val = train_test_split(features_ohe, target, test_size=0.25, random_state=42)

Используем масштабирование 

In [62]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train_sc = scaler.transform(features_train)
features_val_sc = scaler.transform(features_val)

Проверим сбалансированность целевой переменной

In [63]:
df['grant_status'].value_counts()

grant_status
0    3259
1     854
Name: count, dtype: int64

Вижу, что класс 0 очень сильно преобладает над классом 1, при обучении модели буду использовать __class_weight='balanced'__

In [64]:
model = LogisticRegressionCV(solver='liblinear', random_state=12, class_weight='balanced', cv=5)
model.fit(features_train_sc, target_train)
roc_auc_score(target_val, model.predict_proba(features_val_sc)[:, 1])

0.8367610802132905

In [65]:
pd.DataFrame(
    zip(list(features_ohe.columns), list(abs(model.coef_[0]))), 
    columns=['feature', 'importance']
).sort_values(by=['importance'], ascending=False).head(10)

,feature,importance
25,number_of_unsuccessful_grant_1,0.666701
55,number_of_unsuccessful_grant_1_zero,0.655464
299,contract_value_band___see_note_a_D,0.508215
24,number_of_successful_grant_1,0.479549
54,number_of_successful_grant_1_zero,0.469157
300,contract_value_band___see_note_a_E,0.446326
297,contract_value_band___see_note_a_B,0.432043
244,sponsor_code_4D,0.411098
298,contract_value_band___see_note_a_C,0.374524
302,contract_value_band___see_note_a_G,0.330002


In [66]:
model_RF = RandomForestClassifier(random_state=42)

grid_param = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 2, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(
    estimator=model_RF,
    param_grid=grid_param,
    cv=5,
    n_jobs=-1,
)

grid_search.fit(features_train_sc, target_train)
print("Лучшие параметры: ", grid_search.best_params_)

Лучшие параметры:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


Обучим модель случайного леса на наших лучших гиперпараметрах

In [67]:
best_params = grid_search.best_params_

best_RF_model = RandomForestClassifier(**best_params, random_state=42)
best_RF_model.fit(features_train_sc, target_train)

roc_auc_score(target_val, best_RF_model.predict_proba(features_val_sc)[:, 1])

0.8801989564818531

In [68]:
importances = best_RF_model.feature_importances_

feature_importances_df = pd.DataFrame({
    'feature': features_ohe.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

feature_importances_df.head(10)


,feature,importance
25,number_of_unsuccessful_grant_1,0.110003
55,number_of_unsuccessful_grant_1_zero,0.081312
54,number_of_successful_grant_1_zero,0.039046
24,number_of_successful_grant_1,0.035900
50,person_id_1_zero,0.023559
0,rfcd_code_1,0.021719
30,rfcd_code_1_zero,0.021623
40,seo_code_1_zero,0.021458
10,seo_code_1,0.021353
52,dept_no__1_zero,0.020157


__Выводы:__

Для логистической регрессии ROC=0.8367610802132905, самыми важными признаками оказались __number_of_unsuccessful_grant_1__, __number_of_unsuccessful_grant_1_zero__ и __contract_value_band_see_note_a_D__

Для случайного леса ROC=0.8801989564818531, самыми важными признаками оказались __number_of_unsuccessful_grant_1__, __number_of_unsuccessful_grant_1_zero__ и __number_of_successful_grant_1_zero__

Везде вывел топ-3 признака